## Example 2.1:  Prototype of a Shortest Path Algorithm
Initial first version of a shortest path search algorithm

In [ ]:
import itertools
from collections import defaultdict

distances = {
    ('Paris', 'Berlin'): 1000,
    ('Berlin', 'Rome'): 1500,
    ('Berlin', 'Prague'): 350,
    ('Rome', 'Paris'): 1400
}

def find_path(city1, city2):
    all_cities = list(set(itertools.chain.from_iterable(distances.keys())))

    c = {c: float('inf') for c in all_cities}
    prev = {c: None for c in all_cities}
    c[city1] = 0

    u = set(all_cities)

    while u:
        n = min(u, key=lambda x: c[x])
        if c[n] == float('inf'):
            break

        u.remove(n)
        if n == city2:
            break        

        for neighbor in all_cities:
            dist = distances.get((n, neighbor),  distances.get((neighbor, n), float('inf')))
            if dist < float('inf'):
                alt = c[n] + dist
                if alt < c[neighbor]:
                    c[neighbor] = alt
                    prev[neighbor] = n

    if c[city2] == float('inf'):
        return None, None

    path = []
    step = city2
    while step:
        path.insert(0, step)
        step = prev[step]

    return path, c[city2]

city1 = 'Prague'
city2 = 'Paris'
path, distance = find_path(city1, city2)
if path and distance is not None:
    print(f"Shortest path has {distance} km via [{' -> '.join(path)}]")
else:
    print(f"No path found from {city1} to {city2}.")

## Example 2.2: Creating Structure

In [ ]:
def compute_distance(distances: dict, city1: str, city2: str) -> float:
    distance = distances.get((city1, city2))
    if distance is not None:
        return distance
    distance = distances.get((city2, city1))
    if distance is not None:
        return distance
    return float('inf')

def init_state(
        all_cities: list[str], 
        start_city: str) -> tuple[dict, dict]:
    distance_to_city = {c: float('inf') for c in all_cities}
    distance_to_city[start_city] = 0    
    previous_city = {c: None for c in all_cities}
    return distance_to_city, previous_city

def update_route_states(
        distance_to_city: dict, 
        previous_city: dict, 
        destination: str, 
        neighbor: str, 
        current_distance: float) -> tuple[dict, dict]:
    if current_distance < float('inf'):
        alt = distance_to_city[destination] + current_distance
        if alt < distance_to_city[neighbor]:
            distance_to_city[neighbor] = alt
            previous_city[neighbor] = destination    
    return distance_to_city, previous_city

def get_shortest_path(city: str, previous_city: dict) -> list[str]:
    path = []
    step = city
    while step:
        path.insert(0, step)
        step = previous_city[step]
    return path

def find_path(distances: dict, city1: str, city2: str) -> tuple[list[float], float] | tuple[None, None]:
    all_cities = list(set(itertools.chain.from_iterable(distances.keys())))

    distance_to_city, previous_city = init_state(
            all_cities=all_cities, 
            start_city=city1
    )
    city_pool = set(all_cities)

    while city_pool:
        closest_city = min(city_pool, key=lambda x: distance_to_city[x])
        distance_closet_cities = distance_to_city[closest_city]
        if distance_closet_cities == float('inf'):
            break

        city_pool.remove(closest_city)
        if closest_city == city2:
            break        

        for neighbor in all_cities:
            dist = compute_distance(
                distances=distances, 
                city1=closest_city, 
                city2=neighbor
            )
            distance_to_city, previous_city = update_route_states(
                    distance_to_city=distance_to_city, 
                    previous_city=previous_city, 
                    destination=closest_city, 
                    neighbor=neighbor,  
                    current_distance=dist
            )

    if distance_to_city[city2] == float('inf'):
        return None, None

    path = get_shortest_path(city=city2, previous_city=previous_city)
    return path, distance_to_city[city2]


distances = {
    ('Paris', 'Berlin'): 1000,
    ('Berlin', 'Rome'): 1500,
    ('Berlin', 'Prague'): 350,
    ('Rome', 'Paris'): 1400
}
city1 = 'Prague'
city2 = 'Paris'
path, distance = find_path(distances, city1, city2)
if path and distance is not None:
    print(f"Shortest path has {distance} km via [{' -> '.join(path)}]")
else:
    print(f"No path found from {city1} to {city2}.")

## Example 2.3: Separation of State and Function

In [ ]:
class ShortestPathSearch:

    def __init__(self, distances: dict):
        self.distances = distances
        self.all_cities = list(set(itertools.chain.from_iterable(distances.keys())))
        self._distance_to_city = None
        self._previous_city = None

    def init_state(self, start_city: str) -> None:
        self._distance_to_city = {
            c: float('inf') for c in self.all_cities
        }
        self._distance_to_city[start_city] = 0    
        self._previous_city = {
            c: None for c in self.all_cities
        }
    
    def compute_distance(self, 
            city1: str, 
            city2: str
        ) -> float:
        distance = self.distances.get((city1, city2))
        if distance is not None:
            return distance
        distance = self.distances.get((city2, city1))
        if distance is not None:
            return distance
        return float('inf')

    def update_route_states(
        self, 
        destination: str, 
        neighbor: str, 
        current_distance: float
    ) -> None:
        if current_distance < float('inf'):
            alt = self._distance_to_city[destination] + current_distance
            if alt < self._distance_to_city[neighbor]:
                self._distance_to_city[neighbor] = alt
                self._previous_city[neighbor] = destination    

    def get_shortest_path(self, city: str) -> list[str]:
        path = []
        step = city
        while step:
            path.insert(0, step)
            step = self._previous_city[step]
        return path

    def find_path(self, 
        city1: str, 
        city2: str
    ) -> tuple[list[float], float] | tuple[None, None]:
        city_pool = set(self.all_cities)

        self.init_state(start_city=city1)
    
        while city_pool:
            closest_city = min(city_pool, key=lambda x: self._distance_to_city[x])
            distance_closet_cities = self._distance_to_city[closest_city]
            if distance_closet_cities == float('inf'):
                break
    
            city_pool.remove(closest_city)
            if closest_city == city2:
                break        
    
            for neighbor in self.all_cities:
                dist = self.compute_distance(
                        city1=closest_city, 
                        city2=neighbor
                    )
                self.update_route_states(
                        destination=closest_city, 
                        neighbor=neighbor, 
                        current_distance=dist
                )    
    
        if self._distance_to_city[city2] == float('inf'):
            return None, None
    
        path = self.get_shortest_path(city=city2)

        return path, self._distance_to_city[city2]

distances = {
    ('Paris', 'Berlin'): 1000,
    ('Berlin', 'Rome'): 1500,
    ('Berlin', 'Prague'): 350,
    ('Rome', 'Paris'): 1400
}

city1 = 'Prague'
city2 = 'Paris'
path, distance = ShortestPathSearch(distances=distances).find_path(city1, city2)
if path and distance is not None:
    print(f"Shortest path has {distance} km via [{' -> '.join(path)}]")
else:
    print(f"No path found from {city1} to {city2}.")

## Example 2.4: Structuring the Main Function

In [ ]:
import itertools

class ShortestPathSearch:

    def __init__(self, distances: dict):
        self.distances = distances
        self.all_cities = list(set(itertools.chain.from_iterable(distances.keys())))
        self._distance_to_city = None
        self._previous_city = None
        self._city_pool = None

    def init_state(self, start_city: str) -> None:
        self._distance_to_city = {
            c: float('inf') for c in self.all_cities
        }
        self._distance_to_city[start_city] = 0    
        self._previous_city = {
            c: None for c in self.all_cities
        }
        self._city_pool = set(self.all_cities)
    
    def compute_distance(self, 
        city1: str, 
        city2: str
    ) -> float:
        distance = self.distances.get((city1, city2))
        if distance is not None:
            return distance
        distance = self.distances.get((city2, city1))
        if distance is not None:
            return distance
        return float('inf')

    def update_route_states(self, 
        destination: str, 
        neighbor: str, 
        current_distance: float
    ) -> None:
        if current_distance < float('inf'):
            alt = self._distance_to_city[destination] + current_distance
            if alt < self._distance_to_city[neighbor]:
                self._distance_to_city[neighbor] = alt
                self._previous_city[neighbor] = destination    

    def get_shortest_path(self, city: str) -> list[str]:
        path = []
        step = city
        while step:
            path.insert(0, step)
            step = self._previous_city[step]
        return path

    def loop_cities_update_distances(self, closest_city: str) -> None:
        for neighbor in self.all_cities:
            dist = self.compute_distance(
                    city1=closest_city, 
                    city2=neighbor
                )
            self.update_route_states(
                destination=closest_city, 
                neighbor=neighbor, 
                current_distance=dist
            )

    def continue_search_with_closet_city(self, 
        closest_city: str, 
        target_city: str
    ) -> bool:
        distance_closest_city = self._distance_to_city[closest_city]
        if distance_closest_city == float('inf'):
            return False
        self._city_pool.remove(closest_city)
        if closest_city == target_city:
            return False
        return True

    def find_path(self, 
        city1: str, 
        city2: str
    ) -> tuple[list[float], float] | tuple[None, None]:
        
        self.init_state(start_city=city1)
    
        while self._city_pool:
            closest_city = min(self._city_pool, key=lambda x: self._distance_to_city[x])
            if not self.continue_search_with_closet_city(
                closest_city=closest_city,
                target_city=city2
            ):
                break
            self.loop_cities_update_distances(closest_city=closest_city)
    
        if self._distance_to_city[city2] == float('inf'):
            return None, None
        path = self.get_shortest_path(city=city2)
        return path, self._distance_to_city[city2]

distances = {
    ('Paris', 'Berlin'): 1000,
    ('Berlin', 'Rome'): 1500,
    ('Berlin', 'Prague'): 350,
    ('Rome', 'Paris'): 1400
}

city1 = 'Prague'
city2 = 'Paris'
path, distance = ShortestPathSearch(distances=distances).find_path(city1, city2)
if path and distance is not None:
    print(f"Shortest path has {distance} km via [{' -> '.join(path)}]")
else:
    print(f"No path found from {city1} to {city2}.")

## Example 2.5: Black Box Unit Test: Simple Example

In [ ]:
my_test_distances = {
    ('Berlin', 'Amsterdam'): 1,
    ('Amsterdam', 'Paris'): 1,
    ('Berlin', 'Paris'): 25
}

route, distance = ShortestPathSearch(distances=my_test_distances).find_path("Berlin", "Paris")
assert route == ["Berlin", "Amsterdam", "Paris"]
assert distance == 2

## Example 2.6: Black Box Unit Test: Edge Case

In [ ]:
import unittest
route, distance = ShortestPathSearch(distances=my_test_distances).find_path("Berlin", "Berlin")

test_case = unittest.TestCase()
test_case.assertEqual(0, distance)
test_case.assertListEqual(["Berlin"], route)

## Example 2.7: White Box Unit Test

In [ ]:
import unittest

my_test_distances = {
    ('Berlin', 'Amsterdam'): 1
}

test_case = unittest.TestCase()

# check distance from Berlin to Amsterdam
shortest_path = ShortestPathSearch(distances=my_test_distances)
distance = shortest_path.compute_distance(
    "Berlin", "Amsterdam"
)
test_case.assertEqual(1, distance)

# check distance from Amsterdam to Berlin
distance = shortest_path.compute_distance(
    "Amsterdam", "Berlin"
)
test_case.assertEqual(1, distance)

# check distance if any of the two cities is not in the route data
distance = shortest_path.compute_distance(
    "Amsterdam", "missing"
)
test_case.assertEqual(float('inf'), distance)

# check distance both of the two cities is not in the route data
distance = shortest_path.compute_distance(
    "missing", "missing"
)
test_case.assertEqual(float('inf'), distance)